# Test Functions

Here I'll try to compile some Julia funtion for NURBS

## Knot Function
Subroutine to generate a B-spline open knot vector with multiplicity k at the ends (see Sec. 3.3).

For example we will have:
$$
    \begin{split}
        n = 5,\ k = 2  &  \qquad [0\ 0\ 1\ 2\ 3\ 4\ 4]\\
        n = 5,\ k = 3  &  \qquad [0\ 0\ 0\ 1\ 2\ 3\ 3\ 3]\\
        n = 5,\ k = 4  &  \qquad [0\ 0\ 0\ 0\ 1\ 2\ 2\ 2\ 2]
    \end{split}
$$
Where the number of knots $m$, linked with the number of the controll points $n-1$ and the order of the curve $k$ (=degree of the polinomial function +1) whith the following relation:
$$
    m = n + k + 1
$$

#### Variables Description
```C
/**
 * c      = order of the basis function
 * n      = the number of control polygon vertices
 * nplus2 = index of x() for the first occurence of the maximum knot vector value
 * nplusc = maximum value of the knot vector n + c
 * x()    = array containing the knot vector
 */
```

In [2]:
function knot(n::Int64,c::Int64)::Array{Int64}
    x=Int64[];
    nplusc::Int64 = n+c;
    nplus2::Int64 = n+2;
    push!(x, 0);
    for i = 2:nplusc
        if i>c && i<nplus2
            push!(x, last(x)+1);
        else
            push!(x, last(x));
        end
    end
    return x
end

knot (generic function with 1 method)

In [6]:
knot(7,5)

12-element Array{Int64,1}:
 0
 0
 0
 0
 0
 1
 2
 3
 3
 3
 3
 3

## KnotC Function
Subroutine to generate a nonuniform open knot vector proportional to the chord lengths between defining polygon vertices.

#### Variables Description
```C
/**
 * c           = order of the basis function
 * chord       = chord distance between defining polygon vertices
 * csum        = accummulated sum of the chord distances from the first defining polygon vertex
 * maxchord    = sum of the chord distances between defining polygon vertices
 * npts        = the number of defining polygon vertices
 * nplusc      = maximum value of the knot vector -- $n + c$
 * numerator   = numerator of Eq. (5--86)
 * x()         = array containing the knot vector
 * xchord      = x-component of the distance between defining polygon vertices
 * ychord      = y-component of the distance between defining polygon vertices
 */
```

In [ ]:
function knotc(n::Int64, c::Int64)::Array{Float64}
    [...]
end

## Knotu Function
Subroutine to generate a B-spline periodic uniform knot vector (see Sec. 3.3).

#### Variabls Description
```C
/**
 * c      = order of the basis function
 * n      = the number of control polygon vertices
 * nplus2 = index of x() for the first occurence of the maximum knot vector value
 * nplusc = maximum value of the knot vector n + c
 * x()    = array containing the knot vector
 */
```

In [ ]:
function knotu(n::Int64, c::Int64)::Array{Int64}
    x=Int64[];
    nplusc=n+c;
    nplus2=n+2;
    for i=1:nplusc
        push!(x, i-1);
    end
    return x;
end

## Basis Algorithm
Subroutine to generate B-spline basis functions for open uniform knot vectors (see Eqs. (3.2)).

#### Variable Description
```C
/**
 * c       = order of the B-spline basis function
 * d       = first term of the basis function recursion relation
 * e       = second term of the basis function recursion relation
 * npts    = number of control polygon vertices
 * n(,)    = array containing the basis functions 
 *            n(1,1) contains the basis function associated with B1 etc.
 * nplusc  = constant npts + c, maximum number of knot values
 * t       = parameter value
 * tempt() = temporary array
 * x()     = knot vector
 */
```

In [ ]:
function basis(c::Int64, t::Float64, npts::Int64, x::Array{Int64})::Array{Float64}
    tempt = Array{Float64}(36)
    n = Float64[];
    nplusc = npts + c;
    
#    print(knot vector is );
#    for i=1:nplusc
#        print(& &, i, x[i]);
#    end
#    print(t is &, t);
    
    # calculate the first order basis functions n[i][1]
    
    for i=1:nplusc-1
        if (t>=x[i])&&(t<x[i+1])
            temp[i] = 1;
        else
            temp[i] = 0;
        end
    end
    
    # calculate the higher order basis functions
    for k=2:c
        for i=1:nplusc-k
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                d = ((t-x[i])*temp[i])/(x[i+k-1]-x[i]);
            else
                d = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                e = ((x[i+k]-t)*temp[i+1])/(x[i+k]-x[i+1]);
            else
                e = 0;
            end
            temp[i] = d+e;
        end
    end
    
    if t == x[nplusc]    # pick the last point
        temp[npts] = 1;
    end
    
    # put in n array
    for i=1:npts
        push!(n, temp[i]);
    end
    return n;
end

## DBasis Algorithm
Subroutine to generate B-spline basis functions and their derivatives for uniform open knot vectors (see Eqs. (3.2) and (3.29)-(3.32)).

#### Variables Description
```C
/**
 * bl     = first term of the basis function
 * b2     = second term of the basis function
 * c      = order of the B-spline basis function
 * d1(,)  = array containing the derivative of the basis functions
 *           d1(1,1) contains the derivate for B_1 etc.
 * d2(,)  = array containing the derivative of the basis functions
 *           d1(1,1) contains the derivate for B_1 etc.
 * fl     = first term of the first derivative of the basis function
 * f2     = second term of the first derivative of the basis function
 * f3     = third term of the first derivative of the basis function
 * f4     = fourth term of the first derivative of the basis function
 * npts   = number of control polygon vertices
 * n(,)   = array containing the basis functions
 *          n(1,1) contains the basis function for B_1 etc.
 * nplusc = constant npts + c, maximum knot value
 * sl     = first term of the second derivative of the basis function
 * s2     = second term of the second derivative of the basis function
 * s3     = third term of the second derivative of the basis function
 * s4     = fourth term of the second derivative of the basis function
 * t      = parameter value
 * temp() = temporary array
 * x()    = knot vector
 */
```

In [ ]:
function dbasis(c::Int64, t::Float64, npts::Int64, x::Array{Int64})::Tuple{Array{Float64}, Array{Float64}, Array{Float64}}
    
    #inizialization
    
    temp = zeros(36);    # allows for 35 defining polygon vertices
    temp1 = zeros(36);
    temp2 = zeros(36);
    nplusc = npts+c;
    
    # calculate the first order basis functions n[i]
    
    for i=1:nplusc-1
        if (t>=x[i])&&(t<x[i+1])
            temp[i]=1;
        else
            temp[i]=0;
        end
    end
    
    if t==x[nplusc]    # last(x)
        temp[npts] = 1;
    end
    
    # calculate the higher order basis functions
    for k=2:c
        for i=1:nplusc-k
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                b1 = ((t-x[i])*temp[i])/(x[i+k-1]-x[i]);
            else
                b1 = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                b2 = ((x[i+k]-t)*temp[i+1])/(x[i+k]-x[i+1]);
            else
                b2 = 0;
            end
            
            # calculate first derivative
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                f1 = temp[i]/(x[i+k-1]-x[i]);
            else
                f1 = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                f2 = -temp[i+1]/(x[i+k]-x[i+1]);
            else
                f2 = 0;
            end
            if temp1[i] != 0    # if the lower order basis function is zero skip the calculation
                f3 = ((t-x[i])*temp1[i])/(x[i+k-1]-x[i]);
            else
                f3 = 0;
            end
            if temp1[i+1] != 0    # if the lower order basis function is zero skip the calculation
                f4 = ((x[i+k]-t)*temp1[i+1])/(x[i+k]-x[i+1]);
            else
                f4 = 0;
            end
            
            # calculate second derivative
            if temp1[i] != 0    # if the lower order basis function is zero skip the calculation
                s1 = (2*temp1[i])/(x[i+k-1]-x[i]);
            else
                s1 = 0;
            end
            if temp1[i+1] != 0    # if the lower order basis function is zero skip the calculation
                s2 = (-2*temp1[i+1])/(x[i+k]-x[i+1]);
            else
                s2 = 0;
            end
            if temp2[i] != 0    # if the lower order basis function is zero skip the calculation
                s3 = ((t-x[i])*temp2[i])/(x[i+k-1]-x[i]);
            else
                s3 = 0;
            end
            if temp2[i+1] != 0    # if the lower order basis function is zero skip the calculation
                s4 = ((x[i+k]-t)*temp2[i+1])/(x[i+k]-x[i+1]);
            else
                s4 = 0;
            end
            
            temp[i] = b1 + b2;
            temp1[i] = f1 + f2 + f3 + f4;
            temp2[i] = s1 + s2 + s3 + s4;
        end
    end
    
    # prepare output
    for i=1:npts
        push!(n, temp[i]);
        push!(d1, temp1[i]);
        push!(d2, temp2[i]);
    end
    return (n, d1, d2);
end

## DBasisU Algorithm
Subroutine to generate B-spline basis functions and their derivatives for uniform periodic knot vectors (see Eqs. (3.2) and (3.29)-(3.32)).

#### Variables Description
```C
/**
 * b1     = first term of the basis function
 * b2     = second term of the basis function
 * c      = order of the B-spline basis function
 * d1(,)  = array containing the derivative of the basis functions
 *           d1(1,1) contains the derivative of the basis function for Bi etc.
 * d2(,)  = array containing the derivative of the basis functions
 *           d2(1,1) contains the derivative of the basis function for Bi etc.
 * f1     = first term of the first derivative of the basis function
 * f2     = second term of the first derivative of the basis function
 * f3     = third term of the first derivative of the basis function
 * f4     = fourth term of the first derivative of the basis function
 * npts   = number of control polygon vertices
 * n(,)   = array containing the basis functions
 *           n(1,1) contains the basis function for B_1 etc.
 * nplusc = constant npts + c, maximum knot value
 * sl     = first term of the second derivative of the basis function
 * s2     = second term of the second derivative of the basis function
 * s3     = third term of the second derivative of the basis function
 * s4     = fourth term of the second derivative of the basis function
 * t      = parameter value
 * temp() = temporary array
 * x()    = knot vector
 */
```

In [ ]:
function dbasisu(c::Int64, t::Float64, npts::Int64, x::Array{Int64})::tuple{Array{Float64}, Array{Float64}, Array{Float64}}

    #inizialization
    
    temp = zeros(36);    # allows for 35 defining polygon vertices
    temp1 = zeros(36);
    temp2 = zeros(36);
    nplusc = npts+c;
    
    # calculate the first order basis functions n[i]
    
    for i=1:nplusc-1
        if (t>=x[i])&&(t<x[i+1])
            temp[i]=1;
        else
            temp[i]=0;
        end
    end
    
    if t==x[npts+1]    # handle the end specially
        temp[npts] = 1;    # resetting the first order basis functions.
        temp[npts+1]=0;
    end
    
    # calculate the higher order basis functions
    for k=2:c
        for i=1:nplusc-k
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                b1 = ((t-x[i])*temp[i])/(x[i+k-1]-x[i]);
            else
                b1 = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                b2 = ((x[i+k]-t)*temp[i+1])/(x[i+k]-x[i+1]);
            else
                b2 = 0;
            end
            
            # calculate first derivative
            if temp[i] != 0    # if the lower order basis function is zero skip the calculation
                f1 = temp[i]/(x[i+k-1]-x[i]);
            else
                f1 = 0;
            end
            if temp[i+1] != 0    # if the lower order basis function is zero skip the calculation
                f2 = -temp[i+1]/(x[i+k]-x[i+1]);
            else
                f2 = 0;
            end
            if temp1[i] != 0    # if the lower order basis function is zero skip the calculation
                f3 = ((t-x[i])*temp1[i])/(x[i+k-1]-x[i]);
            else
                f3 = 0;
            end
            if temp1[i+1] != 0    # if the lower order basis function is zero skip the calculation
                f4 = ((x[i+k]-t)*temp1[i+1])/(x[i+k]-x[i+1]);
            else
                f4 = 0;
            end
            
            # calculate second derivative
            if temp1[i] != 0    # if the lower order basis function is zero skip the calculation
                s1 = (2*temp1[i])/(x[i+k-1]-x[i]);
            else
                s1 = 0;
            end
            if temp1[i+1] != 0    # if the lower order basis function is zero skip the calculation
                s2 = (-2*temp1[i+1])/(x[i+k]-x[i+1]);
            else
                s2 = 0;
            end
            if temp2[i] != 0    # if the lower order basis function is zero skip the calculation
                s3 = ((t-x[i])*temp2[i])/(x[i+k-1]-x[i]);
            else
                s3 = 0;
            end
            if temp2[i+1] != 0    # if the lower order basis function is zero skip the calculation
                s4 = ((x[i+k]-t)*temp2[i+1])/(x[i+k]-x[i+1]);
            else
                s4 = 0;
            end
            
            temp[i] = b1 + b2;
            temp1[i] = f1 + f2 + f3 + f4;
            temp2[i] = s1 + s2 + s3 + s4;
        end
    end
    
    # prepare output
    for i=1:npts
        push!(n, temp[i]);
        push!(d1, temp1[i]);
        push!(d2, temp2[i]);
    end
    return (n, d1, d2);
end